In [3]:
import sys
import os
from pathlib import Path

# Get absolute path to project root
project_root = Path(os.getcwd()).resolve()
if project_root.name != "ncg87-blockchain_tracker":
    project_root = project_root.parent  # Adjust if running from a subdirectory

sys.path.append(str(project_root))

# Add to Python path if not already there
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from config import Settings

from database import MongoDatabase, MongoInsertOperations, MongoQueryOperations
from database import SQLDatabase, SQLInsertOperations, SQLQueryOperations
from chains import BaseContractProcessor

import json
import web3


In [4]:
sql_db = SQLDatabase()
w3 = web3.Web3(web3.HTTPProvider(Settings.BASE_ENDPOINT))


In [5]:
query = "SELECT * FROM evm_contract_abis WHERE network = 'Base'"
sql_db.cursor.execute(query)
contract_abis = sql_db.cursor.fetchall()


In [6]:
len(contract_abis)

9305

In [7]:
contract_abis[0]

('Base',
 '0x304E5a1Ce122c12D83199689562549866f1BE1c3',
 '[{"inputs": [], "stateMutability": "nonpayable", "type": "constructor"}, {"anonymous": false, "inputs": [{"indexed": true, "internalType": "address", "name": "owner", "type": "address"}, {"indexed": true, "internalType": "int24", "name": "tickLower", "type": "int24"}, {"indexed": true, "internalType": "int24", "name": "tickUpper", "type": "int24"}, {"indexed": false, "internalType": "uint128", "name": "amount", "type": "uint128"}, {"indexed": false, "internalType": "uint256", "name": "amount0", "type": "uint256"}, {"indexed": false, "internalType": "uint256", "name": "amount1", "type": "uint256"}], "name": "Burn", "type": "event"}, {"anonymous": false, "inputs": [{"indexed": true, "internalType": "address", "name": "owner", "type": "address"}, {"indexed": false, "internalType": "address", "name": "recipient", "type": "address"}, {"indexed": true, "internalType": "int24", "name": "tickLower", "type": "int24"}, {"indexed": true, "

In [8]:
contract_list = []

for abi in contract_abis:
    if abi[2] is None:
        continue
    abi_json = json.loads(abi[2])
    contract = w3.eth.contract(abi[1], abi = abi_json)
    contract_list.append(contract)


In [6]:
ERC20_ABI = [
    {
        "constant": True,
        "inputs": [],
        "name": "name",
        "outputs": [{"name": "", "type": "string"}],
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [],
        "name": "symbol",
        "outputs": [{"name": "", "type": "string"}],
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [],
        "name": "decimals",
        "outputs": [{"name": "", "type": "uint8"}],
        "type": "function"
    }
]

In [8]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def process_contract(args):
    i, contract = args
    try:
        token_0 = contract.functions.token0().call()
        token_1 = contract.functions.token1().call()
        token0_contract = w3.eth.contract(address=token_0, abi=ERC20_ABI)
        token1_contract = w3.eth.contract(address=token_1, abi=ERC20_ABI)
        contract_factory = contract.functions.factory().call()
        contract_fee = contract.functions.fee().call()
        contract_token0 = token0_contract.functions.name().call()
        contract_token1 = token1_contract.functions.name().call()
        
        return (i, {
            'address': contract.address,
            'factory': contract_factory,
            'fee': contract_fee,
            'token0_name': contract_token0,
            'token1_name': contract_token1
        })
    except Exception as e:
        print(f"Error processing contract {i}: {e}")
        return (i, None)

# Use ThreadPoolExecutor since we're doing I/O-bound operations
contract_info = []
max_workers = 10  # Adjust this number based on your needs

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Create futures for all contracts
    futures = [executor.submit(process_contract, (i, contract)) 
              for i, contract in enumerate(contract_list)]
    
    # Process completed futures as they finish
    for future in as_completed(futures):
        i, result = future.result()
        if result:
            contract_info.append(result)
            print(f"Processed contract {i}: {result['token0_name']} - {result['token1_name']}")

Error processing contract 2: ("The function 'token0' was not found in this ", "contract's abi.")Error processing contract 3: ("The function 'token0' was not found in this ", "contract's abi.")

Error processing contract 5: ("The function 'token0' was not found in this ", "contract's abi.")
Error processing contract 6: ("The function 'token0' was not found in this ", "contract's abi.")
Error processing contract 8: ("The function 'token0' was not found in this ", "contract's abi.")
Error processing contract 9: ('The abi for this contract contains no function definitions. ', 'Are you sure you provided the correct contract abi?')
Error processing contract 10: ("The function 'token0' was not found in this ", "contract's abi.")
Error processing contract 11: ('The abi for this contract contains no function definitions. ', 'Are you sure you provided the correct contract abi?')
Error processing contract 12: ("The function 'token0' was not found in this ", "contract's abi.")
Error processing con

In [9]:
len(contract_info)

1098

In [15]:
contract_info[5]

{'address': '0x70aCDF2Ad0bf2402C957154f944c19Ef4e1cbAE1',
 'factory': '0x5e7BB104d84c7CB9B682AaC2F3d509f5F406809A',
 'fee': 500,
 'token0_name': 'Wrapped Ether',
 'token1_name': 'Coinbase Wrapped BTC'}

In [16]:
factory_group = {}
for contract in contract_info:
    if contract['factory'] not in factory_group:
        factory_group[contract['factory']] = []
    factory_group[contract['factory']].append(contract)


In [20]:
factory_group.keys()


dict_keys(['0x0BFbCF9fa4f9C56B0F40a671Ad40E0805A091865', '0x5e7BB104d84c7CB9B682AaC2F3d509f5F406809A', '0x33128a8fC17869897dcE68Ed026d694621f6FDfD', '0xc35DADB65012eC5796536bD9864eD8773aBc74C4', '0x38015D05f4fEC8AFe15D7cc0386a126574e8077B', '0x2F0d41f94d5D1550b79A83D2fe85C82d68c5a3ca', '0x0Fd83557b2be93617c9C1C1B6fd549401C74558C', '0xb5620F90e803C7F957A9EF351B8DB3C746021BEa', '0xE6dA85feb3B4E0d6AEd95c41a125fba859bB9d24', '0x3D237AC6D2f425D2E890Cc99198818cc1FA48870', '0xedDef4273518b137CDbcB3a7FA1C6a688303dFe2', '0x9592CD9B267748cbfBDe90Ac9F7DF3c437A6d51B', '0xE82Fa4d4Ff25bad8B07c4d1ebd50e83180DD5eB8', '0xC05A5aA56DF0Dc97D6B9849A06627a079790014f', '0xdC323d16C451819890805737997F4Ede96b95e3e', '0x6e0E8D1703a0D0c1b41Edc56EaED55A40386a441'])

In [21]:
for key in factory_group.keys():
    print(key, len(factory_group[key]))


0x0BFbCF9fa4f9C56B0F40a671Ad40E0805A091865 96
0x5e7BB104d84c7CB9B682AaC2F3d509f5F406809A 137
0x33128a8fC17869897dcE68Ed026d694621f6FDfD 762
0xc35DADB65012eC5796536bD9864eD8773aBc74C4 25
0x38015D05f4fEC8AFe15D7cc0386a126574e8077B 29
0x2F0d41f94d5D1550b79A83D2fe85C82d68c5a3ca 2
0x0Fd83557b2be93617c9C1C1B6fd549401C74558C 14
0xb5620F90e803C7F957A9EF351B8DB3C746021BEa 8
0xE6dA85feb3B4E0d6AEd95c41a125fba859bB9d24 5
0x3D237AC6D2f425D2E890Cc99198818cc1FA48870 13
0xedDef4273518b137CDbcB3a7FA1C6a688303dFe2 1
0x9592CD9B267748cbfBDe90Ac9F7DF3c437A6d51B 1
0xE82Fa4d4Ff25bad8B07c4d1ebd50e83180DD5eB8 1
0xC05A5aA56DF0Dc97D6B9849A06627a079790014f 1
0xdC323d16C451819890805737997F4Ede96b95e3e 2
0x6e0E8D1703a0D0c1b41Edc56EaED55A40386a441 1


In [23]:
factory_group['0x33128a8fC17869897dcE68Ed026d694621f6FDfD']


[{'address': '0x304E5a1Ce122c12D83199689562549866f1BE1c3',
  'factory': '0x33128a8fC17869897dcE68Ed026d694621f6FDfD',
  'fee': 10000,
  'token0_name': 'BACA',
  'token1_name': 'Wrapped Ether'},
 {'address': '0x54Ca06eeDAbA89ded2fE3bFB6B16b8965Fec9977',
  'factory': '0x33128a8fC17869897dcE68Ed026d694621f6FDfD',
  'fee': 10000,
  'token0_name': 'Fed Chain',
  'token1_name': 'Wrapped Ether'},
 {'address': '0x24321402927e526A122C4df7cd4Af4E5d472aab7',
  'factory': '0x33128a8fC17869897dcE68Ed026d694621f6FDfD',
  'fee': 10000,
  'token0_name': 'launchbot',
  'token1_name': 'Wrapped Ether'},
 {'address': '0xA7c0584d125EdE124Fd107c511b0F6263a3b0E68',
  'factory': '0x33128a8fC17869897dcE68Ed026d694621f6FDfD',
  'fee': 10000,
  'token0_name': 'COOCHIE',
  'token1_name': 'Wrapped Ether'},
 {'address': '0xD3ed6D02d14Df6dEe2FdBd5457AE5CFb274E416f',
  'factory': '0x33128a8fC17869897dcE68Ed026d694621f6FDfD',
  'fee': 10000,
  'token0_name': 'USD Coin',
  'token1_name': 'AI Agent Layer'},
 {'address':